## Module 1: Intro

#### RAG with minsearch

In [1]:
import minsearch
import json

from openai import OpenAI

Indexing the documents in minsearch

In [2]:
with open("documents.json") as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}

    result = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5
    )

    return result

In [4]:
def build_prompt(query, search_result):
    prompt_template = """
        You're a course teaching assistant. Answer the question based on the context from the FAQ database.
        Use only the facts from the context when answering the questions.
        QUESTION: {question}
        CONTEXT:
            {context}
    """.strip()

    context = ""
    for doc in search_result:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [14]:
def llm(prompt):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [36]:
query = "how to run kafka?"

In [48]:
def rag(query):
    print("RAG with minsearch")
    
    search_result = search(query)
    prompt = build_prompt(query, search_result)
    answer = llm(prompt)
    
    return answer

In [49]:
%time rag(query)

RAG with minsearch
CPU times: user 57.7 ms, sys: 3.38 ms, total: 61.1 ms
Wall time: 4.78 s


'To run Kafka, you can follow the general steps outlined for running a producer or consumer in the terminal. In your project directory, execute the following command:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the actual name of your JAR file. If you are using Python for producing or consuming messages, ensure that you have set up your virtual environment and installed the required packages from `requirements.txt`.'

#### RAG with Elasticsearch

In [28]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

es_client = Elasticsearch("http://localhost:9200")

Indexing the documents in Elasticsearch

In [25]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [29]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [30]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    result_docs = []
    response = es_client.search(index=index_name, body=search_query)
    
    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs

In [50]:
def rag(query):
    print("RAG with ES")
    
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [51]:
%time rag(query)

RAG with ES
CPU times: user 45.9 ms, sys: 5.96 ms, total: 51.8 ms
Wall time: 4.76 s


"To run a Kafka producer or consumer in the terminal using Java, navigate to the project directory and execute the following command:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nFor running a Python Kafka producer, first create a virtual environment and install the required packages specified in `requirements.txt`. Here are the steps:\n\n1. Create a virtual environment (run only once):\n   ```\n   python -m venv env\n   source env/bin/activate  # On MacOS/Linux\n   ```\n\n   For Windows, use:\n   ```\n   env\\Scripts\\activate\n   ```\n\n2. Install the required packages:\n   ```\n   pip install -r ../requirements.txt\n   ```\n\n3. Activate the virtual environment whenever needed:\n   ```\n   source env/bin/activate  # On MacOS/Linux\n   ```\n\n   For Windows:\n   ```\n   env\\Scripts\\activate\n   ```\n\n4. To deactivate it, run:\n   ```\n   deactivate\n   ``` \n\nMake sure to have Docker images up and running if you'r